# Hierarchical Clustering

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm
from __future__ import division
import numpy as np
# from keras.models import Sequential
# from keras.layers.core import Dense
# from keras.optimizers import SGD
import pandas as pd
from sklearn.model_selection import train_test_split
# import theano
import cv2
from sklearn import preprocessing
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Perform hierarchical clustering on observations (columns)

In [2]:
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, dendrogram, set_link_color_palette
import scipy.spatial.distance as ssd
from faker import Factory

fake = Factory.create()
colorList = []

for i in range(5000):
    colorList.append(fake.hex_color())

set_link_color_palette(colorList)

df = pd.read_csv('y_briefly.csv')
# df = df[df['Yr'] >= 2011]
# df = df[df['AST%'] <= 15]
# df = df[df['Pos'] >= 2]
# df = df[df['Pos'] <= 4]
df.reset_index(inplace=True, drop=True)

df['Player_ID'] = df['Player_ID'].astype(int)

y = np.array(df['OBPM 3-4-5'].values)
pid = np.array(df['Player_ID'].values)
yr = np.array(df['Yr'].values)
images = []
for i in range(len(pid)):
    images.append(cv2.imread('thumbnails/thumbnail_{0}_{1}-{2}.png'.format(str(pid[i]), str(yr[i]-1), str(yr[i])[-2:]), 0))
X = np.array(images)
flat_exes = []
for i in range(len(X)):
    flat_exes.append(X[i].flatten())
X = np.array(flat_exes)

In [3]:
# Create distance matrix on columns using cosine distance
# convert the redundant n*n square matrix form into a condensed nC2 array
dist = ssd.squareform(pdist(X, 'cosine'))
clust = linkage(dist, method='complete')

# fig, ax = plt.subplots(figsize=(20,10))
# _ = dendrogram(clust, color_threshold=1.2, orientation='right', ax=ax)
# _ = plt.xticks(fontsize=10)
# _ = plt.yticks(fontsize=20)

/Users/peternygaard/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  after removing the cwd from sys.path.


In [4]:
R = dendrogram(clust,
                color_threshold=1.2, no_plot=True)

In [5]:
from collections import defaultdict
from matplotlib.colors import rgb2hex, colorConverter

class Clusters(dict):
    def _repr_html_(self):
        html = '<table style="border: 0;">'
        for c in self:
            hx = rgb2hex(colorConverter.to_rgb(c))
            html += '<tr style="border: 0;">' \
            '<td style="background-color: {0}; ' \
                       'border: 0;">' \
            '<code style="background-color: {0};">'.format(hx)
            html += c + '</code></td>'
            html += '<td style="border: 0"><code>' 
            html += repr(self[c]) + '</code>'
            html += '</td></tr>'

        html += '</table>'

        return html

def get_cluster_classes(den, label='ivl'):
    cluster_idxs = defaultdict(list)
    for c, pi in zip(den['color_list'], den['icoord']):
        for leg in pi[1:3]:
            i = (leg - 5.0) / 10.0
            if abs(i - int(i)) < 1e-5:
                cluster_idxs[c].append(int(i))

    cluster_classes = Clusters()
    for c, l in cluster_idxs.items():
        i_l = [den[label][i] for i in l]
        cluster_classes[c] = i_l

    return cluster_classes

lineups = get_cluster_classes(R)

In [6]:
# lineups

In [7]:
thing = []

for ok in range(len(list(lineups.values()))):
    comps = list(lineups.values())[ok]

    for i in comps:
#         thing.append([df['Player'][int(i)], df['Age'][int(i)], ok, df['OBPM'][int(i)], df['OBPM 3-4-5'][int(i)], df['AST%'][int(i)], df['AST% 3-4-5'][int(i)]])
         thing.append([df['Player'][int(i)], df['Pos'][int(i)], df['Yr'][int(i)], df['Age'][int(i)], ok, df['OBPM'][int(i)], df['OBPM 3-4-5'][int(i)], df['AST%'][int(i)], df['AST% 3-4-5'][int(i)], df['FTr'][int(i)], df['STL%'][int(i)], df['ORB%'][int(i)], df['BLK%'][int(i)], df['USG%'][int(i)]])
    loadout = pd.DataFrame(thing)
#     loadout.rename(columns={0:'Player', 1: 'Age', 2: 'Cluster', 3: 'OBPM', 4:'OBPM 3-4-5', 5: 'AST%', 6:'AST% 3-4-5'},inplace=True)
    loadout.rename(columns={0:'Player', 1: 'Pos', 2: 'Year', 3: 'Age', 4: 'Cluster', 5: 'OBPM', 6:'OBPM 3-4-5', 7: 'AST%', 8:'AST% 3-4-5', 9:'FTr', 10:'STL%', 11:'ORB%', 12:'BLK%', 13:'USG%'},inplace=True)
    loadout = loadout.sort_values('OBPM 3-4-5', ascending=False)
    loadout.reset_index(inplace=True, drop=True)
#     print(loadout)
#     print('------------')

dx = pd.DataFrame(thing)
# dx.rename(columns={0:'Player', 1: 'Age', 2: 'Cluster', 3: 'OBPM', 4:'OBPM 3-4-5', 5: 'AST%', 6:'AST% 3-4-5'},inplace=True)
dx.rename(columns={0:'Player', 1: 'Pos', 2: 'Year', 3: 'Age', 4: 'Cluster', 5: 'OBPM', 6:'OBPM 3-4-5', 7: 'AST%', 8:'AST% 3-4-5', 9:'FTr', 10:'STL%', 11:'ORB%', 12:'BLK%', 13:'USG%'},inplace=True)

In [8]:
keys = np.array(list(lineups.keys()))
uniqlo = np.where(keys == 'b')[0][0]
naCl = max(dx['Cluster']) - np.array(dx[dx['Cluster'] == uniqlo].index)[0]

for i in np.array(dx[dx['Cluster'] == uniqlo].index)[1:]:
    dx['Cluster'][i] = naCl + i

/Users/peternygaard/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
dx = dx[~dx.duplicated()]
dx.reset_index(inplace=True, drop=True)
dx[dx['Cluster'] == 979].sort_values('OBPM', ascending=False)

,Player,Pos,Year,Age,Cluster,OBPM,OBPM 3-4-5,AST%,AST% 3-4-5,FTr,STL%,ORB%,BLK%,USG%
4624,Bradley Beal,SG,2017,23,979,4.5,NaN,16.2,NaN,0.259,1.5,2.2,0.7,26.5
4623,Danny Granger,SF,2010,26,979,3.1,NaN,13.6,NaN,0.376,2.0,3.2,1.6,28.7
4625,Kyle Lowry,PG,2011,24,979,3.1,NaN,29.4,NaN,0.310,2.0,3.9,0.6,18.6
4616,Gary Harris,SG,2018,23,979,2.4,NaN,13.0,NaN,0.178,2.6,1.8,0.5,20.7
4619,Jameer Nelson,PG,2012,29,979,1.4,NaN,34.0,NaN,0.145,1.2,1.9,0.2,21.4
4617,Taurean Prince,SF,2018,23,979,-0.7,NaN,13.9,NaN,0.185,1.7,2.3,1.5,22.1
4622,Caris LeVert,3,2017,22,979,-0.7,NaN,13.7,NaN,0.245,1.9,2.0,0.5,16.6
4620,Dion Waiters,SG,2018,26,979,-2.0,NaN,19.8,NaN,0.168,1.3,1.0,0.8,25.0
4626,Andrew Harrison,1,2017,22,979,-2.4,NaN,20.0,NaN,0.539,1.9,1.7,1.3,16.4
4621,Alexey Shved,SG,2014,25,979,-2.7,NaN,14.9,NaN,0.380,1.9,3.4,1.8,20.8


In [347]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error

wheres = []
whats = []

dgenx = dx[dx['OBPM 3-4-5'] >= -9]
# dgenx = dgenx[dgenx['AST%'] < 10]
# dgenx = dgenx[dgenx['Pos'] < 4]
# dgenx = dgenx[dgenx['Yr'] >= 2011]
dgenx.reset_index(inplace=True, drop=True)

X = np.array(dgenx[['Age', 'Cluster', 'OBPM']].values)
y = np.array(dgenx['OBPM 3-4-5'].values)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=71)

model = XGBRegressor(base_score=0.59519, learning_rate=.002, n_estimators=2222)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
score = model.score(X_test, y_test)
print(score)
print(mean_absolute_error(y_test, y_pred))

    #     for j in range(100):
    #         regr = RandomForestRegressor(random_state=j, n_estimators=1000)
    #         regr.fit(X_train, y_train)
    #         predictions = regr.predict(X_test)
    #         if regr.score(X_test, y_test) > 0:
    #             wheres.append([i, j])
    #             whats.append(regr.score(X_test, y_test))
    #             print(i, j, regr.score(X_test, y_test))

    # use if statements to only expand on ones where first result clears a certain threshold

0.5038549296195415
1.2579509136062


In [326]:
# X = np.array(dgenx[['Age', 'Cluster', 'OBPM']].values)
# y = np.array(dgenx['OBPM 3-4-5'].values)

# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=71)

# model = XGBRegressor(base_score=0.59519, learning_rate=.002, n_estimators=2222)
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)
# score = model.score(X_test, y_test)
# print(score)
# print(mean_absolute_error(y_test, y_pred))

# 0.5038549296195415
# 1.2579509136062


# X = np.array(dgenx[['Age', 'Cluster']].values)
# y = np.array(dgenx['AST% 3-4-5'].values)

# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=51)

# model = XGBRegressor(base_score=0.0211395, learning_rate=0.005975, n_estimators=2049)
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)
# score = model.score(X_test, y_test)
# print(score)
# print(mean_absolute_error(y_test, y_pred))

# 0.33896696413169125
# 5.216034874226313


# X = np.array(dgenx[['Cluster']].values)
# y = np.array(dgenx['AST% 3-4-5'].values)

# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=61)

# model = XGBRegressor(base_score=0.0157505, learning_rate=0.00811499, n_estimators=2000)
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)
# score = model.score(X_test, y_test)
# print(score)
# print(mean_absolute_error(y_test, y_pred))

# X = np.array(dgenx[['Cluster']].values)
# y = np.array(dgenx['OBPM 3-4-5'].values)

# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3)

# model = XGBRegressor(base_score=0.025, learning_rate=0.0042265, n_estimators=2000)
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)
# score = model.score(X_test, y_test)
# print(score)
# print(mean_absolute_error(y_test, y_pred))

In [327]:
# X_tract = []

# for g in range(len(X_test)):
#     comsense = dgenx[dgenx['Age'] == X_test[g][0]]
#     comsense = comsense[comsense['Cluster'] == X_test[g][1]]
# #     comsense = comsense[comsense['USG%'] == X_test[g][2]]
# #     comsense = comsense[comsense['FTr'] == X_test[g][3]]
#     X_tract.append(comsense.index[0])

# testers = dgenx.iloc[X_tract]
# testers['Proj. Y'] = model.predict(X_test)
# testers.sort_values('Proj. Y', ascending=False)

In [389]:
future = dx[dx['OBPM 3-4-5'].isna()]
future = future[future['Player'] == 'Jusuf Nurkic']
# future = future[future['AST%'] < 20]
# future = future[future['Pos'] < 4]
future.reset_index(inplace=True, drop=True)

X_future = np.array(future[['Age', 'Cluster', 'OBPM']].values)

future['Proj. Y'] = model.predict(X_future)
future.sort_values('Proj. Y', ascending=False)

,Player,Pos,Year,Age,Cluster,OBPM,OBPM 3-4-5,AST%,AST% 3-4-5,FTr,STL%,ORB%,BLK%,USG%,Proj. Y
3,Jusuf Nurkic,C,2019,24,1529,1.6,NaN,17.9,NaN,0.442,1.7,13.4,4.2,24.7,-0.006332
0,Jusuf Nurkic,C,2018,23,217,-2.6,NaN,11.7,NaN,0.287,1.5,10.0,4.3,26.4,-1.985443
2,Jusuf Nurkic,C,2017,22,1518,-2.3,NaN,13.9,NaN,0.393,1.8,12.3,4.2,23.9,-2.389935
1,Jusuf Nurkic,C,2016,21,1491,-4.1,NaN,12.3,NaN,0.409,2.2,12.9,6.5,27.2,-3.001456


In [39]:
# df_cluster

In [284]:
# df_nocluster

In [246]:
future[future['Player'] == 'Nikola Jokic']

,Player,Pos,Year,Age,Cluster,OBPM,OBPM 3-4-5,AST%,AST% 3-4-5,FTr,STL%,ORB%,BLK%,USG%,Proj. Y
83,Nikola Jokic,5,2016,20,735,2.6,NaN,18.1,NaN,0.317,2.3,11.4,2.3,19.9,20.202913
